# Bar grafieken (Verticaal staafdiagram)
Een kolomgrafiek gebruik je voornamelijk om numerieke waarden te vergelijken tussen verschillende categorieën. Een kolomgrafiek is het duidelijkst als het aantal categorieën klein is. 

Twee uitbreidingen zijn het horizontale staafdiagram (vooral handig bij langere namen van de categorieën) en het gestapelde staafdiagram. Die laatste gebruik je om aan te tonen hoe individuele onderdelen bijdragen aan het geheel. De onderdelen kunnen in absolute waarden worden weergegeven of procentueel

## Download de data
We zullen werken met de Titanic gegevensset die we eerst downloaden

In [3]:
from pathlib import Path
from zipfile import ZipFile
import requests
URL="https://www.kaggle.com/api/v1/datasets/download/yasserh/titanic-dataset"
TITANIC_ZIP = "titanic.zip"
titanic_path = Path(TITANIC_ZIP)
if not titanic_path.exists():
    data = requests.get(URL)
    with open(TITANIC_ZIP,"wb") as f:
        f.write(data.content)
    with open(TITANIC_ZIP, 'rb') as f:
        ZipFile(f).extractall()


## Data verkennen
Alhoewel we die dataset al kennen, gaan we die eerst toch nog eens verkennen

In [ ]:
import pandas as pd
TITANIC_CSV = "Titanic-Dataset.csv"
df = pd.read_csv(TITANIC_CSV)
df.head()

In [ ]:
df.info()

## Bekijk ook de tekstkolommen
Omdat we met categorieën gaan werken, kunnen ook de tekstkolommen interessant zijn

In [ ]:
df.describe(include='all')

## Businessvraag
Vergelijk de overlevingskansen van de passagiers op basis van geslacht en klasse. We hebben de volgende kolommen nodig: Survived en Sex. Van Sex kunnen we een categorie maken. We zien dat er geen ontbrekende waarden zijn.

In [ ]:
df = pd.read_csv(TITANIC_CSV, usecols=['Survived', 'Sex'], dtype={'Survived':pd.BooleanDtype(), 'Sex':'category'})
df.info()

## Klaarmaken van de gegevens
Hoeveel mannen hebben de ramp overleefd? Hoeveel vrouwen hebben de ramp overleefd?

In [ ]:
mannen_overleefd = df.loc[df.Sex == 'male', 'Survived'].sum()
vrouwen_overleefd = df.loc[df.Sex == 'female', 'Survived'].sum()
mannen_overleefd, vrouwen_overleefd 


## Plot de gegevens
Met [plt.bar()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html) kunnen we een staafgrafiek maken. Het eerste argument zijn de labels, het tweede argument bevat de waarden van 'staven'.

Met [plt.text()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.text.html) kunnen we een tekst in de staven zetten. 

In [ ]:
import matplotlib.pyplot as plt

bars = plt.bar(['vrouwen', 'mannen'], [vrouwen_overleefd, mannen_overleefd], width=0.7, color='steelblue')
#plt.bar_label(bars, labels=[f'{int(c)}' for c in bars.datavalues], label_type='edge')
plt.title('Aantal overlevenden bij de ramp met de Titanic')
plt.text(0, vrouwen_overleefd - 10, vrouwen_overleefd, ha="center", color="#FFF", fontsize=8)
plt.text(1, mannen_overleefd - 10, mannen_overleefd, ha="center", color="#FFF", fontsize=8)
plt.show()

## meerdere subplots per klasse
We maken voor elke klasse een aparte subplot. Hiervoor moeten we ook de Pclass kolom inlezen.

In [ ]:
df = pd.read_csv(TITANIC_CSV, usecols=['Survived', 'Sex', 'Pclass'], dtype={'Survived':pd.BooleanDtype(), 'Sex':'category'})
df.info()

## Staafgrafieken voor elke klasse
We hebben voor elk geslacht nu drie waarden nodig. We zullen hiervoor gewone numpy arrays gebruiken

In [ ]:
import numpy as np
mannen_overleefd = np.empty(3)
vrouwen_overleefd = np.empty(3)
for klasse in range(1, 4):
    mannen_overleefd[klasse - 1] = df.loc[(df.Sex == 'male') & (df.Pclass == klasse), 'Survived'].sum()
    vrouwen_overleefd[klasse - 1] = df.loc[(df.Sex == 'female') & (df.Pclass == klasse), 'Survived'].sum()
mannen_overleefd, vrouwen_overleefd

## Maak de plots
We definiëren drie subplots in de figuur. De drie plots delen de y-as zodat ze gemakkelijk te vergelijken zijn. 

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5), sharey=True)
for klasse in range(3):
    ax[klasse].bar(["vrouwen", "mannen"], [vrouwen_overleefd[klasse],mannen_overleefd[klasse]])
    ax[klasse].set_title(f"Aantal overlevenden in klasse {klasse + 1}")
plt.savefig('out.png')
plt.show()